In [1]:
import math
import pandas as pd

# Pricing Models: Pricing Natural Gas Contracts
### Task 2:
You need to create a prototype pricing model that can go through further validation and testing before being put into production.
Eventually, this model may be the basis for fully automated quoting to clients, but for now, the desk will use it with manual oversight to explore options with the client. 

You should write a function that is able to use the data you created previously to price the contract. The client may want to choose multiple dates to inject and withdraw a set amount of gas, so your approach should generalize the explanation from before. Consider all the cash flows involved in the product.

The input parameters that should be taken into account for pricing are:
Injection dates. 
Withdrawal dates.
The prices at which the commodity can be purchased/sold on those dates.
The rate at which the gas can be injected/withdrawn.
The maximum volume that can be stored.
Storage costs.

Write a function that takes these inputs and gives back the value of the contract. You can assume there is no transport delay and that interest rates are zero.
Market holidays, weekends, and bank holidays need not be accounted for. Test your code by selecting a few sample inputs.

In [2]:
import pandas as pd
import math

def gas_storage_contract_value(
    injection_dates,
    injection_volumes,
    withdrawal_dates,
    withdrawal_volumes,
    storage_cost_rate,
    total_vol,
    injection_withdrawal_cost_rate
):
    # Read CSV for price data
    price_data = pd.read_csv("Nat_Gas_forecast.csv", parse_dates=["Dates"])
    price_data.set_index("Dates", inplace=True)  # Set 'Date' as the index

    # Ensure dates are converted to datetime objects
    injection_dates = pd.to_datetime(injection_dates, format="%Y-%m-%d")
    withdrawal_dates = pd.to_datetime(withdrawal_dates, format="%Y-%m-%d")

    # Initialize variables
    volume = 0  # Current stored volume
    total_buy_cost = 0 
    total_sell_value = 0 

    # Combine and sort all relevant dates
    all_dates = sorted(set(injection_dates).union(set(withdrawal_dates)))

    # Loop over all relevant dates
    for current_date in all_dates:
        # Handle injections
        if current_date in injection_dates.values:
            index = injection_dates.get_loc(current_date)
            inject_vol = injection_volumes[index]
            
            # Fetch price for injection
            inject_price = price_data.loc[current_date]
            inject_price = inject_price.iloc[0]

            # Check if there's enough storage capacity for the injection
            if volume + inject_vol <= total_vol:
                volume += inject_vol
                total_buy_cost += inject_vol * inject_price
                total_buy_cost += inject_vol * injection_withdrawal_cost_rate
                print(f"Injected {inject_vol} units of gas on {current_date.strftime('%Y-%m-%d')} at price {inject_price:.2f}")
            else:
                print(f"Insufficient storage capacity on {current_date.strftime('%Y-%m-%d')} for {inject_vol} units")
        
        # Handle withdrawals
        if current_date in withdrawal_dates.values:
            index = withdrawal_dates.get_loc(current_date)
            withdraw_vol = withdrawal_volumes[index]
            
            # Fetch price for withdrawal
            withdraw_price = price_data.loc[current_date]
            withdraw_price = withdraw_price.iloc[0]

            # Check if there's enough gas stored for withdrawal
            if volume >= withdraw_vol:
                volume -= withdraw_vol
                total_sell_value += withdraw_vol * withdraw_price
                total_sell_value -= withdraw_vol * injection_withdrawal_cost_rate
                print(f"Withdrew {withdraw_vol} units of gas on {current_date.strftime('%Y-%m-%d')} at price {withdraw_price:.2f}")
            else:
                print(f"Insufficient stored volume on {current_date.strftime('%Y-%m-%d')} for {withdraw_vol} units")

    # Calculate storage costs based on the entire duration of the contract
    contract_duration_days = (max(withdrawal_dates) - min(injection_dates)).days
    storage_duration_months = math.ceil(contract_duration_days / 30)  # Assume 30-day months
    total_storage_cost = storage_duration_months * storage_cost_rate

    # Calculate the net contract value
    net_value = total_sell_value - total_buy_cost - total_storage_cost

    print(f"Total Buy Cost: {total_buy_cost:.2f}")
    print(f"Total Sell Value: {total_sell_value:.2f}")
    print(f"Total Storage Cost: {total_storage_cost:.2f}")
    print(f"Net Contract Value: {net_value:.2f}")

    return net_value


In [3]:
# Example test case with more favorable pricing
injection_dates = ["2024-01-31", "2024-05-31", "2024-11-30"]
injection_volumes = [1000, 1500, 2000]

withdrawal_dates = ["2024-06-30", "2025-05-31"]
withdrawal_volumes = [1500, 2000]

storage_cost_rate = 1000  # Cost per month
total_vol = 5000  # Max storage capacity
injection_withdrawal_cost_rate = 2  # Cost per unit for injection/withdrawal

# Calculate contract value
contract_value = gas_storage_contract_value(
    injection_dates, 
    injection_volumes, 
    withdrawal_dates, 
    withdrawal_volumes, 
    storage_cost_rate, 
    total_vol, 
    injection_withdrawal_cost_rate
)

print("Final Contract Value:", contract_value)

Injected 1000 units of gas on 2024-01-31 at price 12.60
Injected 1500 units of gas on 2024-05-31 at price 11.40
Withdrew 1500 units of gas on 2024-06-30 at price 11.50
Injected 2000 units of gas on 2024-11-30 at price 12.68
Withdrew 2000 units of gas on 2025-05-31 at price 12.14
Total Buy Cost: 64057.62
Total Sell Value: 34527.59
Total Storage Cost: 17000.00
Net Contract Value: -46530.03
Final Contract Value: -46530.03321040168
